In [1]:
!pip install -U transformers==4.47.0 datasets accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EvalPrediction,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict

# ========== 1. Read dataset ==========
def read_dataset(filepath):
    return pd.read_csv(filepath, on_bad_lines='skip')

# ========== 2. One-hot encode labels ==========
def convert_labels_to_one_hot(dataset, label_column, separator=';'):
    dataset[label_column] = dataset[label_column].astype(str).fillna('')
    labels_list = dataset[label_column].apply(lambda x: [label.strip() for label in x.split(separator) if label.strip()])
    mlb = MultiLabelBinarizer()
    one_hot_labels = mlb.fit_transform(labels_list)
    one_hot_df = pd.DataFrame(one_hot_labels, columns=mlb.classes_)
    return one_hot_df, mlb

# ========== 3. Load data ==========
train_df = read_dataset("/kaggle/input/data-cv/train.csv")
val_df = read_dataset("/kaggle/input/data-cv/val.csv")
test_df = read_dataset("/kaggle/input/data-cv/test.csv")

train_one_hot, mlb = convert_labels_to_one_hot(train_df, "label")
val_one_hot, _ = convert_labels_to_one_hot(val_df, "label")
test_one_hot, _ = convert_labels_to_one_hot(test_df, "label")

train_df = pd.concat([train_df[["text"]], train_one_hot], axis=1)
val_df = pd.concat([val_df[["text"]], val_one_hot], axis=1)
test_df = pd.concat([test_df[["text"]], test_one_hot], axis=1)

# ========== 4. Label mapping ==========
labels = mlb.classes_.tolist()
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# ========== 5. Convert to HF datasets ==========
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)
dataset = DatasetDict({'train': train_dataset, 'val': val_dataset, 'test': test_dataset})

# ========== 6. Tokenizer ==========
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def preprocess_data(examples):
    text = examples["text"]
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
    labels_batch = {k: examples[k] for k in examples if k in labels}
    labels_matrix = np.zeros((len(text), len(labels)))
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    encoding["labels"] = labels_matrix.tolist()
    return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

# ========== 7. pos_weight ==========
label_counts = train_one_hot.sum(axis=0)
total_counts = len(train_one_hot)
pos_weight_np = (total_counts - label_counts) / (label_counts + 1e-6)
pos_weight = torch.tensor(pos_weight_np.values, dtype=torch.float32)

# ========== 8. Load model ==========
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# ========== 9. CustomTrainer ==========
class CustomTrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight
        self.logs = []  # Store logs for each epoch

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight.to(logits.device))
        loss = loss_fn(logits, labels.float())  
        return (loss, outputs) if return_outputs else loss

    def evaluate(self, eval_dataset=None, **kwargs):
        results = super().evaluate(eval_dataset=eval_dataset, **kwargs)

        # Extract the specific metrics
        eval_f1 = results.get('eval_f1', None)
        eval_loss = results.get('eval_loss', None)
        eval_roc_auc = results.get('eval_roc_auc', None)
        eval_accuracy = results.get('eval_accuracy', None)

        # Log results for each epoch
        self.logs.append({
            'Epoch': self.state.epoch,
            'Training Loss': self.state.log_history[-1].get('loss', 0),
            'Validation Loss': eval_loss,
            'F1': eval_f1,
            'Roc Auc': eval_roc_auc,
            'Accuracy': eval_accuracy,
        })

        # Save logs after each epoch
        self.save_logs()

        return results

    def save_logs(self, output_file="training_logs.csv"):
        # Save logs to a CSV file
        logs_df = pd.DataFrame(self.logs)
        logs_df.to_csv(output_file, index=False)

# ========== 10. Training arguments ==========
training_args = TrainingArguments(
    output_dir="./deberta-v3-base-model",
    evaluation_strategy="epoch",           # Evaluate at each epoch
    save_strategy="epoch",                # Save the model at each epoch
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,                 
    weight_decay=0.01,
    load_best_model_at_end=True,          # Load the best model at the end of training
    metric_for_best_model="f1",           # Use F1 score to find the best model
    report_to="none"                     # Disable reporting
)

# ========== 11. Metrics ==========
def multi_label_metrics(preds, labels, threshold=0.5):
    # No need for sigmoid here as preds are already probabilities
    y_pred = np.zeros_like(preds)
    y_pred[preds >= threshold] = 1
    return {
        'f1': f1_score(labels, y_pred, average='micro'),
        'roc_auc': roc_auc_score(labels, y_pred, average='micro'),
        'accuracy': accuracy_score(labels, y_pred)
    }

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    return multi_label_metrics(preds, p.label_ids)

# ========== 12. Trainer ==========
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    pos_weight=pos_weight,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop early if no improvement
)

# ========== 13. Train & Evaluate ==========
trainer.train()

# Save logs to CSV
trainer.save_logs("training_logs.csv")

# ========== 14. Inference ==========
text = "I have 3 years experience in working with Python and write softwares for customers."
encoding = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
encoding = {k: v.to(model.device) for k, v in encoding.items()}

with torch.no_grad():
    outputs = model(**encoding)
logits = outputs.logits
sigmoid = torch.nn.Sigmoid()  # You need this only during inference, not for training or metrics
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros_like(probs)
predictions[probs >= 0.5] = 1
predicted_labels = [id2label[i] for i, val in enumerate(predictions) if val == 1]

print("Predicted labels:", predicted_labels)


2025-05-14 11:26:13.344888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747221973.539923      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747221973.593213      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_35/2589149936.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.345600,0.299033,0.888069,0.932997,0.698074
2,0.272200,0.261012,0.899720,0.941410,0.725744
3,0.258300,0.274960,0.902338,0.944695,0.728897
4,0.219500,0.252136,0.908633,0.946591,0.745359
5,0.214300,0.252054,0.912567,0.948118,0.760420
6,0.184400,0.249450,0.905380,0.949523,0.741856
7,0.177000,0.262951,0.914523,0.950536,0.766025
8,0.153300,0.253831,0.904681,0.951355,0.740105
9,0.149100,0.269238,0.911066,0.950345,0.759720


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Predicted labels: ['Python_Developer', 'Software_Developer']
